In [34]:
import numpy as np
from typing import Union
import math

In [49]:
"""
Given an integer B as represented by two arrays a, b
where B = the product over i of (a_i ^ b_i)
return the number of positive integers 
x less than B such that gcd(x, B) = 1
"""
def phi(a: np.array, b: np.array) -> int:
    
    result = 1
    
    for p,power in zip(a,b):
        
        result *= (p**power) * (1. - 1./p)
    
    return int(result)

In [110]:
a = [2,5]
b = [2,1]
phi(a,b)

8

In [65]:
"""
Given a factor base `factor_base` and an integer a, check whether or not 
a is B-smooth

returns a list `exponents` where each element 
exponents[i] is the exponent of the prime factor_base[i] 
if a is B-smooth

returns an empty array otherwise
"""

def is_B_smooth(a: int, factor_base: np.array) -> Union[np.array, bool]:
    
    exponents = np.zeros_like(factor_base)
    
    for i,p in enumerate(factor_base):
        
        if a == 1:          
            break
        
        while a % p == 0:         
            a = a / p           
            exponents[i] += 1
        
    if a > 1:
        return np.array([])
    
    return exponents

In [66]:
factor_base = [2,3,5,7,11,13,17]
n = 539873
a = 783

is_B_smooth(a**2 % n, factor_base)

array([9, 0, 0, 0, 1, 1, 0])

In [67]:
'''
Given a vector of exponents, return the vector mod-2 
'''
def exponent_vector_mod_2(exponents: np.array) -> np.array:
    
    return exponents % 2

In [68]:
def primes6(n):
    r = [False, True] * (n//2) + [True]
    r[1], r[2] = False, True
    for i in xrange(3, int(1 + n**0.5), 2):
        if r[i]:
            r[i*i::2*i] = [False] * ((n+2*i-1-i*i)/(2*i))
    return r

In [69]:
"""
Given number to factor N pick a B value for factor base
Taken from paper linked in the chat
"""

def choose_B(N: int) -> int:
    L = np.e ** (0.5 * (np.log(N) * np.log(np.log(N)))**0.5)
    return int(np.ceil(L))
    

In [70]:
"""
Given an integer B, use the sieve of Eratosthenes to
find its prime factors 

returns a list of prime factors less than B

"""

def create_factors(n):
    flags = np.ones(n, dtype=bool)
    flags[0] = flags[1] = False
    for i in range(2, int(np.sqrt(n)) + 1):
        if flags[i]:
            flags[i*i::i] = False
    return np.flatnonzero(flags)

In [71]:
B = choose_B(16921456439215439701)

B

651

In [72]:
create_factors(B)

array([  2,   3,   5,   7,  11,  13,  17,  19,  23,  29,  31,  37,  41,
        43,  47,  53,  59,  61,  67,  71,  73,  79,  83,  89,  97, 101,
       103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167,
       173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239,
       241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313,
       317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397,
       401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467,
       479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569,
       571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643,
       647])

In [117]:
'''
Given an integer n, try to factor it into two large primes
Returns a tuple of the factors if able to factor
Returns false if couldn't factor
''' 
def quadratic_sieve(n: int) -> Union[tuple[int, int], bool]:
    
    
    """
    Choose smoothness bound B
    """
    
    B = choose_B(n)
    factor_base = create_factors(B)
    
    
    # represent B as its prime factorization
    # TODO: how to ensure B is prime?
    B_primes = [B]
    B_exp = [1]
    
    print(B, factor_base, B_exp) 


    """
    Find phi(B) + 1 numbers a_i such that 
        b_i = a_i^2 (mod n)
    is B-smooth.
    
    In checking if b_i is B-smooth, we also
    factor b_i.
    
    """
    
    phi_B = phi(factor_base,B_exp)
    
    all_b_exponents = []
    all_a = []
    all_b = []
    
    curr_a = math.ceil(np.sqrt(n))
    
    while len(all_b_exponents) < phi_B + 1:
        
        # find a new entry to add to the list
        curr_b = curr_a**2 % n
        
        curr_exponents = is_B_smooth(curr_b, factor_base)
        
        if (len(curr_exponents) > 0):
            all_a.append(curr_a)
            all_b.append(curr_b)
            all_b_exponents.append(curr_exponents)
        
        curr_a += 1
        
                
    """
    Generate exponent vectors 
    (mod 2) for each b_i
    """
    
    all_exponent_vectors = [ exponent_vector_mod_2(b_exponents) for b_exponents in all_b_exponents ]
    
    
    """
    Find linear combination of these vectors that
    sums to the zero vector
    """
    
    M = np.array(all_exponent_vectors) # matrix of exponent vectors
    
    print(all_a,all_b_exponents,M)
    zero_vector = np.zeros(M.shape[1])
    
    try:
        x = np.linalg.solve(M, zero_vector)
    except np.linalg.LinAlgError:
        x = np.zeros(M.shape[0])
        x[0] = 1
    
    
    
    """
    Multiply the corresponding a_i for the above 
    combination and call the result (mod n) 'a'
    
    Multiply the corresponding b_i for the above 
    combination and call the result (mod n) 'b'
    
    """
    
    a = int(np.prod(all_a * x))
    b = int(np.prod(all_b * x))
    
    factor = np.gcd(a - b, n)
    
    print(factor)
    if (factor != n and factor != 1):
        return factor
    
    return 
    
    
    
    

In [118]:
quadratic_sieve(539873)

19 [ 2  3  5  7 11 13 17] [1]
[735, 750] [array([5, 0, 0, 0, 1, 0, 0]), array([0, 0, 0, 0, 3, 0, 1])] [[1 0 0 0 1 0 0]
 [0 0 0 0 1 0 1]]
539873
